In [1]:
import glob, pandas as pd, numpy as np, re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from tqdm import tqdm

In [2]:
!pip install bertopic -qq --target=/kaggle/working/site-packages
import sys
sys.path.append('/kaggle/working/site-packages')

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.21.0, which is not installed.
dask-cudf 21.10.1 requires cupy-cuda114, which is not installed.
beatrix-jupyterlab 3.1.7 requires google-cloud-bigquery-storage, which is not installed.
thinc 8.0.17 requires typing-extensions<4.2.0,>=3.7.4.1; python_version < "3.8", but you have typing-extensions 4.4.0 which is incompatible.
tfx-bsl 1.9.0 requires tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,!=2.8.*,<3,>=1.15.5, but you have tensorflow 2.6.4 which is incompatible.
tensorflow 2.6.4 requires h5py~=3.1.0, but you have h5py 3.7.0 which is incompatible.
tensorflow 2.6.4 requires numpy~=1.19.2, but you have numpy 1.21.6 which is incompatible.
tensorflow 2.6.4 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
tenso

In [3]:
from bertopic import BERTopic

In [4]:
sws = stopwords.words("english") + ["n't",  "'s", "'ve"]

In [5]:
import pandas as pd
docs = []
df = pd.read_csv('../input/feedback-prize-english-language-learning/train.csv')
for fl in tqdm(df['full_text']):
    word_tokens = word_tokenize(fl)
    txt = " ".join([w for w in word_tokens if not w.lower() in sws])
    docs.append(txt)

100%|██████████| 3911/3911 [00:13<00:00, 279.92it/s]


In [6]:
topic_model = BERTopic(n_gram_range=(1, 3), top_n_words=5, verbose=True)
topics, probs = topic_model.fit_transform(docs)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/123 [00:00<?, ?it/s]

2022-10-24 03:06:30,892 - BERTopic - Transformed documents to Embeddings
2022-10-24 03:06:56,281 - BERTopic - Reduced dimensionality
2022-10-24 03:06:56,419 - BERTopic - Clustered reduced embeddings


In [7]:
tm_meta = topic_model.get_topic_info()
tm_meta.to_csv("topic_model_metadata.csv", index=False)
display(tm_meta)

pred_topics = pd.DataFrame()
pred_topics["id"] = df['text_id']
pred_topics["topic"] = topics
pred_topics['prob'] = probs
pred_topics.to_csv("topic_model_feedback.csv", index=False)
pred_topics

,Topic,Count,Name
0,-1,335,-1_life_people_want_make
1,0,298,0_online_classes_home_students
2,1,222,1_school_hours_day_time
3,2,207,2_attitude_positive_positive attitude_life
4,3,177,3_technology_people_use_use technology
5,4,172,4_accomplish_something_always_idle
6,5,162,5_career_young_age_young age
7,6,160,6_first_impression_impressions_change
8,7,146,7_years_high school_high_school
9,8,140,8_group_working_work_alone


,id,topic,prob
0,0016926B079C,0,1.000000
1,0022683E9EA5,44,1.000000
2,00299B378633,31,1.000000
3,003885A45F42,-1,0.000000
4,0049B1DF5CCC,36,0.827854
...,...,...,...
3906,FFD29828A873,17,0.786362
3907,FFD9A83B0849,8,1.000000
3908,FFDC4011AC9C,38,1.000000
3909,FFE16D704B16,18,1.000000


In [8]:
topic_model.save("feedback3_topic_model")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
